In [16]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import numpy as np

# Load data from CSV files
base_rate = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\3_base_rate.csv")
territory_factor = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\4_Territory_factor_Rating_Area_Factors_USAA_Group_Owners_Forms.csv")
fire_protection_class_construction_factor = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\5_9_Protection_construction_factor_owner.csv")
form_factor = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\6_form_factor.csv")
tier_factor = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\7_tier_factor.csv")
amount_of_insurance_deductible = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\8_amount_of_insurance_factor.csv")
age_of_home_table = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\10_age_of_home.csv")
insured_age_table = pd.read_csv(("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\15_Insured_age_factor.csv"))
roof_factor_table = pd.read_csv(("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\22_roof_factor.csv"))
sqftxnum_stories_table = pd.read_csv(("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\23_sq_ftxnum_stories.csv"))
sqftxnum_stories_table['Max sq ft'] = sqftxnum_stories_table['Max sq ft'].replace('>10000', 15000).astype(int)
num_bathrooms_table = pd.read_csv(("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\24_Num_of_Bathrooms.csv"))
foundation_type_table = pd.read_csv(("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\25_Foundation_Type.csv"))
garage_table = pd.read_csv(("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\26_Garage.csv"))
home_protector_coverage_table = pd.read_csv(("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\27_home_protector_coverage.csv"))
coverage_amount_table = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\28_coverage.csv")
marital_status_table = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\29_marital_status.csv")
fireplaces_table = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\34_num_fireplaces.csv")


def amount_of_insurance_deductible_factor(amount_of_insurance_df, amount_of_insurance, deductible, aep):
    # Define the specific perils to filter
    perils = [
        "Other Perils", "Fire", "Other Wind and Hail", "Theft", "Water",
        "Liability", "Hurricane", "Severe Thunderstorm", "Winter Storm", "Earthquake"
    ]
    
    # Initialize the result DataFrame with perils as columns and a single row for the values
    result = pd.DataFrame(columns=perils, index=[0])
    
    # Filter the dataframe based on the given amount_of_insurance
    filtered_df = amount_of_insurance_df[(amount_of_insurance_df["Amount of Insurance"] == amount_of_insurance) & (amount_of_insurance_df["Peril"].isin(perils))]
    
    # Iterate through each peril to retrieve the appropriate value based on the deductible and aep columns
    for peril in perils:
        # Filter the row for the current peril
        peril_row = filtered_df[filtered_df["Peril"] == peril]
        
        # Check if the peril_row is non-empty
        if not peril_row.empty:
            # First try to retrieve the value from the deductible column
            value = peril_row[deductible].values[0] if deductible in peril_row.columns else np.nan
            
            # If the deductible value is NaN, try the aep column
            if pd.isna(value) and aep in peril_row.columns:
                value = peril_row[aep].values[0]
            
            # If both deductible and aep are NaN, take the value from the "10%" column
            if pd.isna(value) and "10%" in peril_row.columns:
                value = peril_row["10%"].values[0]
            
            # Assign the final value to the result DataFrame
            result[peril].iloc[0] = value
    
    # Adjust the sixth row based on the amount_of_insurance conditions
    if amount_of_insurance in ["$10,000", "$50,000", "$100,000", "$150,000", "$200,000", "$250,000", "$300,000"]:
        result.iloc[0, 5] *= 0.8796
    elif amount_of_insurance in ["$350,000", "$400,000", "$450,000", "$500,000"]:
        result.iloc[0, 5] *= 1.1489
    elif amount_of_insurance in ["$600,000", "$750,000", "$1,000,000", "$1,500,000", "$2,000,000", "$3,000,000"]:
        result.iloc[0, 5] *= 1.5144

    return result


def roof_factor(roof_factor_df, roof_type, impact_resistant_class, roof_age):
    # Define the specific perils to filter
    perils = [
        "Other Perils", "Fire", "Other Wind and Hail", "Theft", "Water",
        "Liability", "Hurricane", "Severe Thunderstorm", "Winter Storm", "Earthquake"
    ]
    
    # Initialize the result DataFrame with perils as columns and a single row for the values
    result = pd.DataFrame(columns=perils, index=[0])
    
    # Filter the dataframe based on the given amount_of_insurance
    filtered_df = roof_factor_df[(roof_factor_df["Roof Type"] == roof_type) & (roof_factor_df["Impact Resistant Class Code"] == impact_resistant_class) & (roof_factor_df["Peril"].isin(perils))]
    
    # Iterate through each peril to retrieve the appropriate value based on the deductible and aep columns
    for peril in perils:
        # Filter the row for the current peril
        peril_row = filtered_df[filtered_df["Peril"] == peril]
        
        # Check if the peril_row is non-empty
        if not peril_row.empty:
            # First try to retrieve the value from the deductible column
            value = peril_row[roof_age].values[0] if roof_age in peril_row.columns else np.nan
                       
            # Assign the final value to the result DataFrame
            result[peril].iloc[0] = value

    return result

def sqftxnum_stories(sqftxnum_stories_df, sqft, num_stories):
    # Define the specific perils to filter
    perils = [
        "Other Perils", "Fire", "Other Wind and Hail", "Theft", "Water",
        "Liability", "Hurricane", "Severe Thunderstorm", "Winter Storm", "Earthquake"
    ]
    
    # Initialize the result DataFrame with perils as columns and a single row for the values
    result = pd.DataFrame(columns=perils, index=[0])
    
    # Filter the dataframe based on the given amount_of_insurance
    if sqft > 10000:
        filtered_df = sqftxnum_stories_df[sqftxnum_stories_df['Min sq ft'] == 10000]
    else:
        filtered_df = sqftxnum_stories_df[(sqftxnum_stories_df['Min sq ft'] <= sqft) & (sqftxnum_stories_df['Max sq ft'] >= sqft)]
    
    # Iterate through each peril to retrieve the appropriate value based on the deductible and aep columns
    for peril in perils:
        # Filter the row for the current peril
        peril_row = filtered_df[filtered_df["Peril"] == peril]
        
        # Check if the peril_row is non-empty
        if not peril_row.empty:
            # First try to retrieve the value from the deductible column
            value = peril_row[num_stories].values[0] if num_stories in peril_row.columns else np.nan
                       
            # Assign the final value to the result DataFrame
            result[peril].iloc[0] = value

    return result


def garage(garage_df, garage_size, garage_type):
    # Define the specific perils to filter
    perils = [
        "Other Perils", "Fire", "Other Wind and Hail", "Theft", "Water",
        "Liability", "Hurricane", "Severe Thunderstorm", "Winter Storm", "Earthquake"
    ]
    
    # Initialize the result DataFrame with perils as columns and a single row for the values
    result = pd.DataFrame(columns=perils, index=[0])
    
    # Filter the dataframe based on the given amount_of_insurance
    filtered_df = garage_df[(garage_df["Garage Size"] == garage_size) & (garage_df["Peril"].isin(perils))]
    
    # Iterate through each peril to retrieve the appropriate value based on the deductible and aep columns
    for peril in perils:
        # Filter the row for the current peril
        peril_row = filtered_df[filtered_df["Peril"] == peril]
        
        # Check if the peril_row is non-empty
        if not peril_row.empty:
            # First try to retrieve the value from the deductible column
            value = peril_row[garage_type].values[0] if garage_type in peril_row.columns else np.nan
                       
            # Assign the final value to the result DataFrame
            result[peril].iloc[0] = value

    return result



def calculate_insurance_premium(rating_area_value, construction_type, protection_class, form, tier, amount_of_insurance_df, amount_of_insurance, 
                                deductible, aep, age_of_home, insurance_claim_history, insured_age, 
                                roof_factor_df, roof_type, impact_resistant_class, roof_age, 
                                sqftxnum_stories_df, sqft, num_stories, num_bathrooms, 
                                foundation_type, garage_df, garage_size, garage_type, coverage_amount, 
                                marital_status, insurance_to_value, early_quote_discount, num_fireplaces):
    # Filter the row for the specified rating area in df_rating_area
    rating_area_row = territory_factor[territory_factor['Rating Area'] == rating_area_value]
    
    # Filter the row for the specified construction type and protection class in df_fire_protection
    fire_protection_class_construction_factor_row = fire_protection_class_construction_factor[
        (fire_protection_class_construction_factor['Construction Type'] == construction_type) & 
        (fire_protection_class_construction_factor['Protection Class'] == protection_class)]
    
    form_factor_row = form_factor[form_factor['Form'] == form]
    tier_factor_row = tier_factor[tier_factor['Tier'] == tier]
    age_of_home_row = age_of_home_table[age_of_home_table['Dwelling Age'] == age_of_home]
    insured_age_row = insured_age_table[insured_age_table['Insured Age'] == insured_age]
    num_bathrooms_row = num_bathrooms_table[num_bathrooms_table['Number of Bathrooms'] == num_bathrooms]
    foundation_type_row = foundation_type_table[foundation_type_table['Foundation'] == foundation_type]
    home_protector_coverage_row = home_protector_coverage_table[home_protector_coverage_table['Form'] == form]
    coverage_amount_row = coverage_amount_table[coverage_amount_table['Coverage Amount'] == coverage_amount]
    marital_status_row = marital_status_table[marital_status_table['Status'] == marital_status]
    fireplaces_row = fireplaces_table[fireplaces_table['Number of Fireplaces'] == num_fireplaces]

    rating_area_values = rating_area_row.drop(columns="Rating Area").iloc[0]
    fire_protection_class_construction_factor_values = fire_protection_class_construction_factor_row.drop(columns=["Construction Type", "Protection Class"]).iloc[0]
    form_factor_values = form_factor_row.drop(columns="Form").iloc[0]
    tier_factor_values =  tier_factor_row.drop(columns=["Tier", "Other Optional Coverages"]).iloc[0]
    amount_of_insurance_deductible_factor_values = amount_of_insurance_deductible_factor(amount_of_insurance_df, amount_of_insurance, deductible, aep)
    age_of_home_values = age_of_home_row.drop(columns="Dwelling Age").iloc[0]
    insured_age_values = insured_age_row.drop(columns="Insured Age").iloc[0]
    roof_factor_values = roof_factor(roof_factor_df, roof_type, impact_resistant_class, roof_age)
    sqftxnum_stories_values = sqftxnum_stories(sqftxnum_stories_df, sqft, num_stories)
    num_bathrooms_values = num_bathrooms_row.drop(columns="Number of Bathrooms").iloc[0]
    foundation_type_values = foundation_type_row.drop(columns="Foundation").iloc[0]
    garage_values = garage(garage_df, garage_size, garage_type)
    home_protector_coverage_values = home_protector_coverage_row.drop(columns="Form").iloc[0]
    coverage_amount_values = coverage_amount_row.drop(columns=["Coverage Amount", "Optional Coverages"]).iloc[0]
    marital_status_values = marital_status_row.drop(columns="Status").iloc[0]
    fireplaces_values = fireplaces_row.drop(columns="Number of Fireplaces").iloc[0]
    

    # Calculating final values by multiplying rating area, protection factors, and transposed DataFrame values
    result = 0.98 * rating_area_values * fire_protection_class_construction_factor_values * form_factor_values * tier_factor_values * amount_of_insurance_deductible_factor_values * age_of_home_values * insured_age_values* roof_factor_values* sqftxnum_stories_values * num_bathrooms_values * foundation_type_values * garage_values * home_protector_coverage_values * coverage_amount_values * marital_status_values * insurance_to_value * early_quote_discount * fireplaces_values * base_rate.iloc[0]
    
    

    # Adjust result based on insurance_claim_history
    if insurance_claim_history == "Clean":
        result *= 0.85
    elif insurance_claim_history == "Previous claims":
        result *= 1

    dataframe = result.T

    dataframe  = dataframe .drop(dataframe.index[-1])

    return dataframe

    

# GUI for the insurance premium calculation
def show_gui():
    def calculate_premium():
        try:
            rating_area_value = int(rating_area_entry.get())
            construction_type = construction_type_combo.get()
            protection_class = int(protection_class_entry.get())
            form = form_combo.get()
            tier = int(tier_entry.get())
            amount_of_insurance_df = amount_of_insurance_deductible
            amount_of_insurance = amount_of_insurance_combo.get()
            deductible = deductible_combo.get()
            aep = aep_combo.get()
            age_of_home = age_of_home_entry.get()
            insurance_claim_history = insurance_claim_history_combo.get()
            insured_age = insured_age_entry.get()
            roof_factor_df = roof_factor_table
            roof_type = roof_type_combo.get()
            impact_resistant_class = int(impact_resistant_class_combo.get())
            roof_age = roof_age_entry.get()
            sqftxnum_stories_df = sqftxnum_stories_table
            sqft = int(sqft_entry.get())
            num_stories = num_stories_combo.get()
            num_bathrooms = num_bathrooms_combo.get()
            foundation_type = foundation_type_combo.get()
            garage_df = garage_table
            garage_size = garage_size_combo.get()
            garage_type = garage_type_combo.get()
            coverage_amount = coverage_amount_combo.get()
            marital_status = marital_status_combo.get()
            insurance_to_value = float(insurance_to_value_entry.get())
            early_quote_discount = float(early_quote_discount_entry.get())
            num_fireplaces = num_fireplaces_combo.get()
            expense_fee = float(expense_fee_entry.get())


            
            # Calculate the result
            result = calculate_insurance_premium(rating_area_value, construction_type, protection_class, form, tier, 
                                         amount_of_insurance_df, amount_of_insurance, deductible, aep, age_of_home, insurance_claim_history, insured_age, 
                                         roof_factor_df, roof_type, impact_resistant_class, roof_age, 
                                         sqftxnum_stories_df, sqft, num_stories, num_bathrooms, 
                                         foundation_type, garage_df, garage_size, garage_type, 
                                         coverage_amount, marital_status, insurance_to_value, 
                                         early_quote_discount, num_fireplaces)
            
            result = result.values.sum() + expense_fee
            
            if result:
                result_label.config(text=f"Calculated Premium: ${result:.2f}")
            else:
                messagebox.showerror("Error", "Invalid inputs. Please check your entries.")
        except ValueError:
            messagebox.showerror("Error", "Please enter valid numbers for Rating Area, Protection Class, and Tier.")
    
    # Set up the window
    window = tk.Tk()
    window.title("Homeowner Insurance Premium Calculator")
    window.geometry("1000x800")
    
    # Input fields
    tk.Label(window, text="Rating Area (1-3501)").grid(row=0, column=0, padx=10, pady=10)
    rating_area_entry = tk.Entry(window)
    rating_area_entry.insert(0, "701")  # Default value
    rating_area_entry.grid(row=0, column=1)
    
    tk.Label(window, text="Construction Type").grid(row=1, column=0, padx=10, pady=10)
    construction_type_combo = ttk.Combobox(window, values=["Masonry Veneer", "Frame", "Masonry"])
    construction_type_combo.set("Masonry Veneer")  # Default value
    construction_type_combo.grid(row=1, column=1)
    
    tk.Label(window, text="Protection Class (1-10)").grid(row=2, column=0, padx=10, pady=10)
    protection_class_entry = tk.Entry(window)
    protection_class_entry.insert(0, "3")  # Default value
    protection_class_entry.grid(row=2, column=1)
    
    tk.Label(window, text="Form").grid(row=3, column=0, padx=10, pady=10)
    form_combo = ttk.Combobox(window, values=["HO-3R", "HO-9R"])
    form_combo.set("HO-3R")  # Default value
    form_combo.grid(row=3, column=1)
    
    tk.Label(window, text="Tier (5-68)").grid(row=4, column=0, padx=10, pady=10)
    tier_entry = tk.Entry(window)
    tier_entry.insert(0, "30")  # Default value
    tier_entry.grid(row=4, column=1)
    
    
    tk.Label(window, text="Amount of insurance").grid(row=5, column=0, padx=10, pady=10)
    amount_of_insurance_combo = ttk.Combobox(window, values=["$10,000", "$50,000", "$100,000",
                                                             "$150,000", "$200,000", "$250,000",
                                                             "$300,000", "$350,000", "$400,000",
                                                             "$450,000", "$500,000", "$600,000", "$750,000",
                                                             "$1,000,000", "$1,500,000", "$2,000,000",
                                                             "$3,000,000"])
    amount_of_insurance_combo.set("$200,000")  # Default value
    amount_of_insurance_combo.grid(row=5, column=1)
    
    tk.Label(window, text="Deductible").grid(row=6, column=0, padx=10, pady=10)
    deductible_combo = ttk.Combobox(window, values=["$500", "$1000", "$2000", "$5000", "$10000"])
    deductible_combo.set("$500")  # Default value
    deductible_combo.grid(row=6, column=1)
    
    tk.Label(window, text="Annual Exceedence Probability (Hurricane)").grid(row=7, column=0, padx=10, pady=10)
    aep_combo = ttk.Combobox(window, values=["1%", "2%", "3%", "5%"])
    aep_combo.set("2%")  # Default value
    aep_combo.grid(row=7, column=1)

    tk.Label(window, text="Dwelling Age (0-99 or 100+)").grid(row=8, column=0, padx=10, pady=10)
    age_of_home_entry = tk.Entry(window)
    age_of_home_entry.insert(0, "10") # Default value
    age_of_home_entry.grid(row=8, column=1)

    tk.Label(window, text="Insurance Claim History").grid(row=9, column=0, padx=10, pady=10)
    insurance_claim_history_combo = ttk.Combobox(window, values=["Clean", "Previous claims"])
    insurance_claim_history_combo.set("Clean")  # Default value
    insurance_claim_history_combo.grid(row=9, column=1)

    tk.Label(window, text="Insured Age (18-99 or 100+)").grid(row=10, column=0, padx=10, pady=10)
    insured_age_entry = tk.Entry(window)
    insured_age_entry.insert(0, "40") # Default value
    insured_age_entry.grid(row=10, column=1)

    tk.Label(window, text="Roof Type").grid(row=11, column=0, padx=10, pady=10)
    roof_type_combo = ttk.Combobox(window, values=[ 'Aluminum', 'Asbestos', 'Cloth', 'Concrete Tile', 'Composition Shingle', 
    'Clay Tile', 'Composition Over Wood', 'Fiberglass Shingle', 'Fiber Cement', 
    'Resin Formed Shingle', 'Foam', 'Metal', 'None', 'Other', 'Plastic', 
    'Reinforced Plastic', 'Slate', 'Tar', 'Unknown', 'Wood Shake', 'No Data'])
    roof_type_combo.set("Fiberglass Shingle")  # Default value
    roof_type_combo.grid(row=11, column=1)

    tk.Label(window, text="Impact Resistance Class Code").grid(row=12, column=0, padx=10, pady=10)
    impact_resistant_class_combo = ttk.Combobox(window, values = [0, 3, 4])
    impact_resistant_class_combo.set(0)  # Default value
    impact_resistant_class_combo.grid(row=12, column=1)

    tk.Label(window, text="Roof age (0-29 or 30+)").grid(row=13, column=0, padx=10, pady=10)
    roof_age_entry = tk.Entry(window)
    roof_age_entry.insert(0, "9") # Default value
    roof_age_entry.grid(row=13, column=1)

    tk.Label(window, text="Building area in sqft").grid(row=0, column=2, padx=10, pady=10)
    sqft_entry = tk.Entry(window)
    sqft_entry.insert(0, "1800") # Default value
    sqft_entry.grid(row=0, column=3)

    tk.Label(window, text="Number of stories").grid(row=1, column=2, padx=10, pady=10)
    num_stories_combo = ttk.Combobox(window, values=["1", "1.5", "2", "2.5", "3+"])
    num_stories_combo.set("1")  # Default value
    num_stories_combo.grid(row=1, column=3)

    tk.Label(window, text="Number of bathrooms").grid(row=2, column=2, padx=10, pady=10)
    num_bathrooms_combo = ttk.Combobox(window, values=["1 or fewer", "2", "3", "4", "5+"])
    num_bathrooms_combo.set("2")  # Default value
    num_bathrooms_combo.grid(row=2, column=3)

    tk.Label(window, text="Foundation type").grid(row=3, column=2, padx=10, pady=10)
    foundation_type_combo = ttk.Combobox(window, values=["Crawlspace", "Pier", "Slab", 
                                                         "Basement <25% finished", "Basement 25-49% finished",
                                                         "Basement 50-74% finished", "Basement 75-99% finished",
                                                         "Basement 100% finished"])
    foundation_type_combo.set("Slab")  # Default value
    foundation_type_combo.grid(row=3, column=3)

    tk.Label(window, text="Garage size").grid(row=4, column=2, padx=10, pady=10)
    garage_size_combo = ttk.Combobox(window, values=["1 Car", "1.5 Car", "2 Car", "2.5 Car", "3+ Car"])
    garage_size_combo.set("2 Car")  # Default value
    garage_size_combo.grid(row=4, column=3)

    tk.Label(window, text="Garage type").grid(row=5, column=2, padx=10, pady=10)
    garage_type_combo = ttk.Combobox(window, values=["Attached", "Basement", "Built-In", "Carport", "Detached", "None"])
    garage_type_combo.set("Attached")  # Default value
    garage_type_combo.grid(row=5, column=3)

    tk.Label(window, text="Coverage Amount").grid(row=6, column=2, padx=10, pady=10)
    coverage_amount_combo = ttk.Combobox(window, values=["2% of Coverage A", "5% of Coverage A", 
                                                     "10% of Coverage A", "15% of Coverage A", 
                                                     "25% of Coverage A", "50% of Coverage A",
                                                     "75% of Coverage A", "100% of Coverage A"])
    coverage_amount_combo.set("10% of Coverage A")  # Default value
    coverage_amount_combo.grid(row=6, column=3)

    tk.Label(window, text="Marital status").grid(row=7, column=2, padx=10, pady=10)
    marital_status_combo = ttk.Combobox(window, values=["Single", "Married"])
    marital_status_combo.set("Single")  # Default value
    marital_status_combo.grid(row=7, column=3)

    tk.Label(window, text="Insurance-to-Value percent, 0.95 to 1 in an increment of 0.0025").grid(row=8, column=2, padx=10, pady=10)
    insurance_to_value_entry = tk.Entry(window)
    insurance_to_value_entry.insert(0, '0.95')  # Default value
    insurance_to_value_entry.grid(row=8, column=3)

    tk.Label(window, text="Early Quote discount").grid(row=9, column=2, padx=10, pady=10)
    early_quote_discount_entry = tk.Entry(window)
    early_quote_discount_entry.insert(0, '0.85')  # Default value
    early_quote_discount_entry.grid(row=9, column=3)

    tk.Label(window, text="Number of Fireplaces").grid(row=10, column=2, padx=10, pady=10)
    num_fireplaces_combo = ttk.Combobox(window, values=["0", "1", "2", "3+"])
    num_fireplaces_combo.set("1")  # Default value
    num_fireplaces_combo.grid(row=10, column=3)

    tk.Label(window, text="Expense Fee").grid(row=11, column=2, padx=10, pady=10)
    expense_fee_entry = tk.Entry(window)
    expense_fee_entry.insert(0, "235")  # Default value
    expense_fee_entry.grid(row=11, column=3)

    # Calculate button and result display
    calculate_button = tk.Button(window, text="Calculate Premium", command=calculate_premium)
    calculate_button.grid(row=14, column=0, columnspan=8, pady=20)
    
    
    result_label = tk.Label(window, text="Calculated Premium: $0.00")
    result_label.grid(row=15, column=0, columnspan=8, pady=10)
    
    # Run the window
    window.mainloop()

# Show the GUI
show_gui()
